In [1]:
from data_preprocess import read_and_normalize_train_data, read_and_normalize_test_data
import numpy as np
import gc
import time

/home/shauk/.conda/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_rows, img_cols = 224, 224
batch_size = 32
nb_epoch = 2

In [3]:
x_train, y_train, driver_id, unique_drivers = read_and_normalize_train_data(img_rows, img_cols, color_type=3)

Restore train from cache!


In [4]:
x_train = np.transpose(x_train, [0,2,3,1])

In [5]:
print(x_train.shape, x_train.nbytes)
print(y_train.shape, y_train.nbytes)

(22424, 224, 224, 3) 13501759488
(22424, 10) 1793920


In [6]:
import tensorflow as tf
from keras.utils import plot_model
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, Flatten

In [7]:
base_model = ResNet50(include_top=False, input_shape=(224,224,3))

In [8]:
resnet50_output = base_model.output
predictions = Dense(10, activation='softmax')(Flatten()(resnet50_output))
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
from sklearn.model_selection import KFold

In [10]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [18]:
gc.get_referrers(y_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [ ]:

for train_index, test_index in kf.split(x_train):
    x = x_train[train_index]
    y = y_train[train_index]
    del x_train
    del y_train
    print(gc.collect())
    print("... to sleep 15 s")
    time.sleep(15)
    
    model.fit(x, y, batch_size=batch_size, epochs=nb_epoch,
             verbose=1, validation_split=0.2, shuffle=True)
    
    del(x)
    del(y)
    gc.collect()
    print("... to read train data")
    x_train, y_train, driver_id, unique_drivers = read_and_normalize_train_data(img_rows, img_cols, color_type=3)

62812
... to sleep 15 s
Train on 16144 samples, validate on 4037 samples
Epoch 1/2
14272/16144 [=========================>....] - ETA: 6s - loss: 1.3385

In [14]:
x = np.arange(2400)
x = x.reshape(100, 24)

In [16]:
x = x.transpose()

[ 3  4  6  7 10 14 15 17 19 20 22 23]
[ 0  1  2  5  8  9 11 12 13 16 18 21]
